In [0]:
"""Importing all the relevant libraries we need."""

import pandas as pd # For data frames
import numpy as np # For math and efficient arrays
import matplotlib.pyplot as plt # For plotting and visualizing data

# So plots appear in this notebook
%matplotlib inline

In [0]:
# Import the data from csv
rain_data = pd.read_csv('https://raw.githubusercontent.com/Mvalentino92/MachineLearning/master/weatherAUS.csv')
rain_data.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RISK_MM
count,141556.000000,141871.000000,140787.000000,81350.000000,74377.000000,132923.000000,140845.000000,139563.000000,140419.000000,138583.000000,128179.000000,128212.000000,88536.000000,85099.000000,141289.000000,139467.000000,142193.000000
mean,12.186400,23.226784,2.349974,5.469824,7.624853,39.984292,14.001988,18.637576,68.843810,51.482606,1017.653758,1015.258204,4.437189,4.503167,16.987509,21.687235,2.360682
std,6.403283,7.117618,8.465173,4.188537,3.781525,13.588801,8.893337,8.803345,19.051293,20.797772,7.105476,7.036677,2.887016,2.720633,6.492838,6.937594,8.477969
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-7.200000,-5.400000,0.000000
25%,7.600000,17.900000,0.000000,2.600000,4.900000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.900000,1010.400000,1.000000,2.000000,12.300000,16.600000,0.000000
50%,12.000000,22.600000,0.000000,4.800000,8.500000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.600000,1015.200000,5.000000,5.000000,16.700000,21.100000,0.000000
75%,16.800000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.400000,1020.000000,7.000000,7.000000,21.600000,26.400000,0.800000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000,40.200000,46.700000,371.000000


In [0]:
print('There are',rain_data.shape[0],'rows and',rain_data.shape[1],'columns')

There are 142193 rows and 24 columns


In [0]:
# Dropping the Date which seems unimportant, and the RISK_MM. 
dropped_features = ['Date','RISK_MM']
features = rain_data.drop(dropped_features,axis=1)
features = features.drop('RainTomorrow',axis=1)
label = rain_data['RainTomorrow']

In [0]:
# Split the numerical and the categorical data
features_dtypes = features.dtypes
num_bool = features_dtypes != 'object'
cat_bool = features_dtypes == 'object'
cols = features.columns
num_features = features[cols[num_bool]]
cat_features = features[cols[cat_bool]]

In [0]:
# Check to make sure they were split correctly
print('Numerical types:\n',num_features.dtypes)
print('\n\nCatergorical types:\n',cat_features.dtypes)

Numerical types:
 MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustSpeed    float64
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
dtype: object


Catergorical types:
 Location       object
WindGustDir    object
WindDir9am     object
WindDir3pm     object
RainToday      object
dtype: object


In [0]:
total_rows = features.shape[0]
for cfeat in cat_features:
    print(cfeat,'has',np.sum(cat_features[cfeat].isna())/total_rows*100,'percent nan values')

print()
for nfeat in num_features:
    print(nfeat,'has',np.sum(num_features[nfeat].isna())/total_rows*100,'percent nan values')

Location has 0.0 percent nan values
WindGustDir has 6.561504434114197 percent nan values
WindDir9am has 7.041837502549352 percent nan values
WindDir3pm has 2.6569521706413113 percent nan values
RainToday has 0.9887969168665124 percent nan values

MinTemp has 0.44798267143952236 percent nan values
MaxTemp has 0.2264527789694289 percent nan values
Rainfall has 0.9887969168665124 percent nan values
Evaporation has 42.78902618272348 percent nan values
Sunshine has 47.692924405561456 percent nan values
WindGustSpeed has 6.5193082641198945 percent nan values
WindSpeed9am has 0.948007285872019 percent nan values
WindSpeed3pm has 1.8495987847503041 percent nan values
Humidity9am has 1.247600092831574 percent nan values
Humidity3pm has 2.5388028946572616 percent nan values
Pressure9am has 9.85561877166949 percent nan values
Pressure3pm has 9.832410878172624 percent nan values
Cloud9am has 37.73533155640573 percent nan values
Cloud3pm has 40.15246882757942 percent nan values
Temp9am has 0.635755

In [0]:
drop_nans = ['Sunshine','Cloud9am','Cloud3pm','Evaporation']
num_features = num_features.drop(drop_nans,axis=1)
dropped_features.extend(drop_nans)
num_features.columns

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am',
       'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am',
       'Pressure3pm', 'Temp9am', 'Temp3pm'],
      dtype='object')

In [0]:
master_bool = np.full((total_rows,),False,dtype=bool)
for cfeat in cat_features:
    master_bool = np.any([master_bool,cat_features[cfeat].isna()],axis=0)
print('Would lose',np.sum(master_bool)/total_rows*100,'percent of data')

Would lose 12.998530166745198 percent of data


In [0]:
# Losing twelve percent is fine, so let's take these rows out of everything first (cat,num,labels)
# First let's flip the master_bool, since we want to actually keep the thins that are currently False!
master_bool = np.logical_not(master_bool)
cat_features = cat_features[master_bool]
num_features = num_features[master_bool]
label = label[master_bool]

In [0]:
# Alright, now we are ready to Onehot encode
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(sparse=False)
cat_X = onehot.fit_transform(cat_features)

# Now we're going to split the categorical features into train and test data
from sklearn.model_selection import train_test_split

# Create seed to split the same way for each piece of data
seed = 23
train_cat_X, test_cat_X = train_test_split(cat_X,test_size=0.20,random_state=seed)

In [0]:
from sklearn.impute import SimpleImputer
# Now let's split the numerical features so we can use an imputer.
# Fit on train, transform on train/test
train_num_features, test_num_features = train_test_split(num_features,test_size=0.20,random_state=seed)

# Create the imputer
imputer = SimpleImputer(strategy='mean')
imputer.fit(train_num_features)

# Imputers return matrices, so save as matrices now
train_num_X = imputer.transform(train_num_features)
test_num_X = imputer.transform(test_num_features)

In [0]:
# Finally let's split the label into train and test. But first, let's convert these 1 and 0
# 1 for yes, and 0 for no.
y = (label == 'Yes').astype(int)
train_y, test_y = train_test_split(y,test_size=0.20,random_state=seed)

In [0]:
# Finally we're going to combine all categorical with the numerical for each train and test
train_X = np.hstack((train_num_X,train_cat_X))
test_X = np.hstack((test_num_X,test_cat_X))

In [0]:
print('Train features shape:',train_X.shape)
print('Test features shape:',test_X.shape)
print('Train labels shape:',train_y.shape)
print('Test labels shape:',test_y.shape)

Train features shape: (98968, 109)
Test features shape: (24742, 109)
Train labels shape: (98968,)
Test labels shape: (24742,)


In [0]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

In [0]:
# Finally, let's use a naive logistic regression and see how it does!
from sklearn.linear_model import LogisticRegression
lreg = LogisticRegression(solver='newton-cg',max_iter=1000,random_state=seed)
lreg.fit(train_X,train_y)
lreg.score(test_X,test_y)

0.8568426157950044

In [0]:
rand_y = np.random.randint(0,2,size=test_y.size)
np.sum(rand_y == test_y)/test_y.size

0.5054563091100154